# Perfil de comércio

## Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

## Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


## Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


## Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.PERIOD_SECONDS=12


### Perfil de comércio

Escolher país e anos de interesse.

In [122]:
country_of_interest = comtrade.m49_mozambique
years_of_interest = comtrade.year_range(2021, 2022)
print(comtrade.COUNTRY_CODES[country_of_interest], years_of_interest)

Mozambique 2021


### Global exports and imports

In [123]:
cols = ['reporterDesc','partnerDesc','flowDesc','primaryValueFormated']
global_trade = comtrade.get_global_stats(country_of_interest, years_of_interest)
global_trade[cols].sort_index()


reporterDesc partnerDesc flowDesc primaryValueFormated
refYear flowCode                                                       
2021    M          Mozambique       World   Import     8,622,696,043.56
        X          Mozambique       World   Export     5,111,685,900.99

## Principais parceiros comerciais
### Principais destinos das exportações

In [124]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 500)

top_partners = comtrade.top_partners(country_of_interest,years_of_interest,flowCode='X', 
                                    global_stats=global_trade,rank_filter=10)
top_partners.sort_values(by=['refYear','flowCode','rank'], ascending=[True,True,True]).style.format({'perc': '{:,.2%}'.format})

## Principais produtos exportados

In [125]:
export_products, df = comtrade.top_commodities(country_of_interest,0,years_of_interest, flowCode='X',
                                           motCode=0,rank_filter=5,
                                           pco_cols=['reporterDesc','partnerDesc','refYear','rank','cmdCode','cmdDesc',
                                                     'flowCode','primaryValue','primaryValueFormated'],
                                           global_stats=global_trade)


In [126]:
cols = ['cmdCode','cmdDesc','primaryValueFormated','perc']
export_products[cols].style.format({'perc': '{:,.2%}'.format})

### Principais produtos por parceiros principais

In [127]:
export_products, df = comtrade.top_commodities(country_of_interest,None,years_of_interest, flowCode='X',
                                           motCode=0, 
                                             partner_first=True,
                                           global_stats=global_trade, 
                                           rank_filter=3)
df = df[df.partnerCode!=0]

In [128]:
import pandas as pd
# show more lines
pd.set_option('display.max_rows', 2500)

export_products[['cmdDesc','primaryValue','perc']].reset_index().head(10).style.format({'perc': '{:,.2%}'.format})

,reporterDesc,partnerDesc,flowCode,refYear,rank,cmdDesc,primaryValue,perc
0,Mozambique,Afghanistan,X,2021,1,Plastics and articles thereof,8079.000000,0.00%
1,Mozambique,Afghanistan,X,2021,2,"Fruit and nuts, edible; peel of citrus fruit or melons",6186.400000,0.00%
2,Mozambique,Angola,X,2021,1,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",308127.750000,0.01%
3,Mozambique,Angola,X,2021,2,"Printed books, newspapers, pictures and other products of the printing industry; manuscripts, typescripts and plans",272032.850000,0.01%
4,Mozambique,Angola,X,2021,3,Plastics and articles thereof,122923.620000,0.00%
5,Mozambique,Argentina,X,2021,1,Animal originated products; not elsewhere specified or included,1460.000000,0.00%
6,Mozambique,Argentina,X,2021,2,"Furniture; bedding, mattresses, mattress supports, cushions and similar stuffed furnishings; lamps and lighting fittings, n.e.c.; illuminated signs, illuminated name-plates and the like; prefabricated buildings",505.190000,0.00%
7,Mozambique,"Asia, not elsewhere specified",X,2021,1,Aluminium and articles thereof,144564.700000,0.00%
8,Mozambique,"Asia, not elsewhere specified",X,2021,2,"Trees and other plants, live; bulbs, roots and the like; cut flowers and ornamental foliage",10834.540000,0.00%
9,Mozambique,"Asia, not elsewhere specified",X,2021,3,"Ores, slag and ash",5544.700000,0.00%


In [129]:
# label for the percentage of a commodity in the total trade of a country
cmd_in_country = 'perc_cmd_for_partner'
# label for the percentage of a country in the total trade of a commodity
country_in_cmd = 'perc_partner_for_cmd'
# Total value of commodities per HS code
df['sum_cmd'] = df.groupby(['reporterDesc','refYear','flowCode','cmdCode'])['primaryValue'].transform('sum')  
# Total value of commodities per partner
df['sum_partner'] = df.groupby(['reporterDesc','refYear','flowCode','partnerCode'])['primaryValue'].transform('sum')
# rank of commodity per HS code
df['rank_cmd'] = df.groupby(['reporterDesc','refYear','flowCode'])['sum_cmd'].rank(ascending=False,method='dense')
df['rank_cmd'] = df['rank_cmd'].astype(int)
# value of trade as percentage of total same HS code
df[country_in_cmd] = df['primaryValue']/df['sum_cmd']
# value of trade as percentage of total same partner
df[cmd_in_country] = df['primaryValue']/df['sum_partner']

# rank of partner per HS code
df['rank_cmd_partner'] = df.groupby(['reporterDesc','refYear','flowCode','cmdCode'])['primaryValue'].rank(ascending=False,method='dense')
df['rank_cmd_partner'] = df['rank_cmd_partner'].astype(int)
# rank of commodity per partner
df['rank_partner_cmd'] = df.groupby(['reporterDesc','refYear','flowCode','partnerCode'])['primaryValue'].rank(ascending=False,method='dense')
df['rank_partner_cmd'] = df['rank_partner_cmd'].astype(int)

# global rank of partner
df['rank_partner'] = df.groupby(['reporterDesc','refYear','flowCode'])['sum_partner'].rank(ascending=False,method='dense')
df['rank_partner'] = df['rank_partner'].astype(int)



### Principais produtos e principais parceiros

In [130]:
show_cols = ['reporterDesc','refYear','flowDesc','rank_cmd','cmdDesc','rank_cmd_partner','rank_partner','partnerDesc','primaryValueFormated',cmd_in_country,country_in_cmd]
sort_list = ['refYear','flowCode','rank_cmd','rank_cmd_partner']
rank_cut = 5
rank_filter = (df['rank_cmd'] <= rank_cut) & (df['rank_cmd_partner'] <= rank_cut)

df[rank_filter].sort_values(by=sort_list)[show_cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_cmd_partner,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
1579,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",1,1,South Africa,"623,155,865.72",72.86%,28.31%
1464,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2,2,India,"578,305,515.42",72.00%,26.27%
882,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,7,Republic of Korea,"190,353,794.47",98.47%,8.65%
255,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,4,China,"139,905,599.80",28.59%,6.36%
1158,Mozambique,2021,Export,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",5,3,Netherlands,"135,251,228.91",24.08%,6.15%
1165,Mozambique,2021,Export,2,Aluminium and articles thereof,1,3,Netherlands,"408,356,074.50",72.72%,36.19%
2033,Mozambique,2021,Export,2,Aluminium and articles thereof,2,5,United Kingdom,"354,618,126.67",95.26%,31.43%
1537,Mozambique,2021,Export,2,Aluminium and articles thereof,3,8,Singapore,"109,367,719.74",72.65%,9.69%
758,Mozambique,2021,Export,2,Aluminium and articles thereof,4,9,Italy,"95,211,869.74",70.59%,8.44%
1716,Mozambique,2021,Export,2,Aluminium and articles thereof,5,10,Spain,"37,164,627.70",35.19%,3.29%


## Principais parceiros comerciais e principais produtos

In [121]:
show_cols = ['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','rank_partner_cmd','rank_cmd','cmdDesc','primaryValueFormated',cmd_in_country,country_in_cmd]
sort_list = ['refYear','flowCode','rank_partner','rank_partner_cmd']
rank_cut = 5
rank_filter = (df['rank_partner'] <= rank_cut) & (df['rank_partner_cmd'] <= rank_cut)

df[rank_filter].sort_values(by=sort_list)[show_cols].style.format({cmd_in_country: '{:,.2%}'.format, country_in_cmd: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,rank_partner_cmd,rank_cmd,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
2216,Angola,2021,Export,1,China,1,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","20,022,774,116.55",99.82%,63.93%
5170,Angola,2021,Export,1,China,2,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","11,961,705.37",0.06%,25.51%
4696,Angola,2021,Export,1,China,3,5,Wood and articles of wood; wood charcoal,"11,645,905.65",0.06%,23.44%
890,Angola,2021,Export,1,China,4,13,"Ores, slag and ash","6,616,586.45",0.03%,66.65%
1100,Angola,2021,Export,1,China,5,18,Aluminium and articles thereof,"966,966.75",0.00%,17.60%
4743,Angola,2021,Export,2,India,1,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","2,907,767,269.85",96.66%,9.28%
3057,Angola,2021,Export,2,India,2,2,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin","100,217,986.40",3.33%,6.52%
1864,Angola,2021,Export,2,India,3,3,"Ships, boats and floating structures","106,755.20",0.00%,0.01%
1254,Angola,2021,Export,2,India,4,6,"Salt; sulphur; earths, stone; plastering materials, lime and cement","70,361.90",0.00%,0.15%
586,Angola,2021,Export,2,India,5,73,Pulp of wood or other fibrous cellulosic material; recovered (waste and scrap) paper or paperboard,"46,527.16",0.00%,99.32%


In [45]:
sort_list = ['reporterDesc','refYear','flowCode','cmdCode']
sort_order = [True,True,True,True]
groupby_list = ['reporterDesc','cmdCode','flowCode','refYear']

sort_list_cmd = ['refYear','flowCode','cmdCode']
sort_order_cmd = [True,True,True]
groupby_list_cmd = ['refYear']

show_cols = ['cmdDesc','primaryValueFormated','rank']
pco = df.set_index(['reporterDesc','refYear','flowCode','cmdCode'])

pco['rank'] = pco.groupby(level=['reporterDesc','refYear','flowCode'])['primaryValue'].rank(method='dense',ascending=False)
pco[show_cols].sort_values([]).head(20)

cmdDesc  \
reporterDesc refYear flowCode cmdCode                                                      
Angola       2020    X        32       Tanning or dyeing extracts; tannins and their ...   
                              73                                  Iron or steel articles   
                              67       Feathers and down, prepared; and articles made...   
                              76                          Aluminium and articles thereof   
                              63       Textiles, made up articles; sets; worn clothin...   
                              76                          Aluminium and articles thereof   
                              63       Textiles, made up articles; sets; worn clothin...   
                              87       Vehicles; other than railway or tramway rollin...   
                              23       Food industries, residues and wastes thereof; ...   
                              65                              Headgear and parts thereof   
                              89                    Ships, boats and floating structures   
                              67       Feathers and down, prepared; and articles made...   
                              69                                        Ceramic products   
                              67       Feathers and down, prepared; and articles made...   
                              06       Trees and other plants, live; bulbs, roots and...   
                              09                            Coffee, tea, mate and spices   
                              08       Fruit and nuts, edible; peel of citrus fruit o...   
                              08       Fruit and nuts, edible; peel of citrus fruit o...   
                              67       Feathers and down, prepared; and articles made...   
                              97           Works of art; collectors' pieces and antiques   

                                      primaryValueFormated    rank  
reporterDesc refYear flowCode cmdCode                               
Angola       2020    X        32                      9.99  2170.0  
                              73                  6,827.88   783.0  
                              67                     69.17  1939.0  
                              76                    707.17  1348.0  
                              63                  1,312.28  1178.0  
                              76              1,955,920.74   121.0  
                              63                    224.55  1654.0  
                              87                    972.71  1255.0  
                              23                     44.09  2024.0  
                              65                 15,223.76   627.0  
                              89             51,677,030.54    36.0  
                              67                     43.23  2027.0  
                              69                     11.23  2164.0  
                              67                     67.36  1947.0  
                              06                     13.14  2154.0  
                              09                     86.47  1901.0  
                              08                  6,222.51   800.0  
                              08                    809.24  1308.0  
                              67                    815.63  1305.0  
                              97                     53.67  1990.0

## Obter os dados

In [4]:

import ipywidgets as widgets
from IPython.display import display
import comtrade

rank_filter = 5  # número de importações mais relevantes
years = "2021"
partnerCode = comtrade.m49_angola # 
flowCode="M"

# select year with widget from list range(2000,2022)

yearsWidget = widgets.SelectMultiple(
    options=list(range(2000,2022)),
    description='Ano:',
    disabled=False,
)

countries = [("China", comtrade.m49_china),("Macau",comtrade.m49_macau) , ("Hong Kong", comtrade.m49_hong_kong)] + comtrade.PLP_TUPLES_REVERSE
# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
reporterCodeWidget = widgets.Dropdown(
    options=countries,
    description='Parceiro A:',
    disabled=False,
)

# select country widget from list Angola, Brazil, Cabo Verde, Guiné Bissau, Equatorial Guinea, Mozambique, Portugal, São Tomé e Príncipe, Timor-Leste
partnerCodeWidget = widgets.Dropdown(
    options=countries,
    description='Parceiro B:',
    disabled=False,
)

# select flowCode widget from list M=Importações, X=Exportações
flowCodeWidget = widgets.Dropdown(
    options=[("Importações", "M"), ("Exportações", "X"),("Ambos", "M,X")],
    value="M",
    description='Fluxo:',
    disabled=False, 
)



### Escolher ano, parceiro e fluxo (import/export) para análise 

Pode escolher-se mais que um ano.

In [8]:
from ipywidgets import VBox

items = [reporterCodeWidget, partnerCodeWidget,yearsWidget]
box = VBox(children=items)
# display vertically
box


In [110]:
yearsList = ",".join(list(map(str,yearsWidget.value)))
partnerA = reporterCodeWidget.value
partnerB = partnerCodeWidget.value
print(yearsList, partnerA, partnerB)

NameError: name 'yearsWidget' is not defined

## Análise de simetrias nos dados

In [109]:
cols = ['reporterDesc','partnerDesc','refYear','flowDesc','primaryValueFormated','primaryValue']

In [108]:
flowCode = 'M'
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )
#df_import_rep_a.info()

NameError: name 'partnerA' is not defined

In [9]:
print(f"Importações de produtos de {comtrade.COUNTRY_CODES.get(partnerA)} por {comtrade.COUNTRY_CODES.get(partnerB)} reportados por {comtrade.COUNTRY_CODES.get(partnerA)}")
df_import_rep_a[cols]

Importações de produtos de China por China reportados por China


,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,China,China,2022,Import,"122,883,424,107.00",1.228834e+11
1,China,China,2021,Import,"156,820,006,224.00",1.568200e+11
2,China,China,2020,Import,"125,264,219,826.00",1.252642e+11
3,China,China,2019,Import,"129,792,198,042.00",1.297922e+11
4,China,China,2018,Import,"146,221,550,788.00",1.462216e+11
5,China,China,2017,Import,"132,369,371,926.00",1.323694e+11
6,China,China,2016,Import,"128,059,540,969.00",1.280595e+11
7,China,China,2015,Import,"143,369,308,743.00",1.433693e+11
8,China,China,2014,Import,"144,880,436,618.00",1.448804e+11
9,China,China,2013,Import,"157,540,887,639.00",1.575409e+11


In [10]:
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )

https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=&partner2Code=0&cmdCode=TOTAL&flowCode=M&customsCode=C00&subscription-key=HIDDEN


/Users/jrc/develop/cipf-comtrade/comtrade.py:276: UserWarning: Server returned HTTP Status: 429
  warnings.warn(f"Server returned HTTP Status: {str(resp.status_code)}",)
/Users/jrc/develop/cipf-comtrade/comtrade.py:279: UserWarning: Server returned error: Rate limit is exceeded. Try again in 9 seconds.
  warnings.warn(f"Server returned error: {message}")


In [11]:
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000000][cols].sort_values(by=['refYear','primaryValue'], ascending=False).head(100)

TypeError: 'NoneType' object is not subscriptable

In [91]:
# invertemos

print(yearsList, partnerA, partnerB)
print(f"Exportações de produtos de {comtrade.COUNTRY_CODES.get(partnerB)} para {comtrade.COUNTRY_CODES.get(partnerA)} reportados por {comtrade.COUNTRY_CODES.get(partnerB)}")

flowCode = 'X'
df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode='TOTAL',
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=None,
                     period=yearsList,
                     motCode = 0,
                     timeout=30,
                     echo_url=True
                     )
# df_import_rep_b.info()

2018,2019,2020,2021 24 156
Exportações de produtos de China para Angola reportados por China
https://comtradeapi.un.org/data/v1/get//C/A/HS?reporterCode=156&period=2018%2C2019%2C2020%2C2021&partnerCode=24&cmdCode=TOTAL&flowCode=X&customsCode=C00&subscription-key=HIDDEN


In [92]:
df_import_rep_a[cols]

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,Angola,China,2018,Import,"2,331,088,265.89",2.331088e+09
4,Angola,China,2019,Import,"2,026,243,881.71",2.026244e+09
11,Angola,China,2020,Import,"1,434,674,061.72",1.434674e+09
15,Angola,China,2021,Import,"1,688,904,641.34",1.688905e+09


In [93]:
df_import_rep_b[cols]

,reporterDesc,partnerDesc,refYear,flowDesc,primaryValueFormated,primaryValue
0,China,Angola,2018,Export,"2,253,137,107.00",2.253137e+09
1,China,Angola,2019,Export,"2,055,727,417.00",2.055727e+09
2,China,Angola,2020,Export,"1,748,092,204.00",1.748092e+09
3,China,Angola,2021,Export,"2,491,545,681.00",2.491546e+09
